In [1]:
import pandas as pd
import numpy as np
import sys
!pip install mecab-python3
import MeCab
import fasttext
#!pip install gensim
from gensim.models.wrappers.fasttext import FastText
import time

In [11]:
## 基本形で分かち書きをする関数

def analysis(text):
    # tagger = MeCab.Tagger("-Ochasen")
    tagger = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    tagger.parse("")
    node = tagger.parseToNode(text)
    word = ""
    pre_feature = ""
    while node:
         # 名詞、形容詞、動詞、形容動詞であるかを判定する。
        isUsed = "名詞" in node.feature
        isUsed = "形容詞" in node.feature or isUsed
        isUsed = "動詞" in node.feature or isUsed
        isUsed = "形容動詞" in node.feature or isUsed
         # 以下に該当する場合は除外する。（ストップワード）
        isUsed = (not "代名詞" in node.feature) and isUsed
        isUsed = (not "助動詞" in node.feature) and isUsed
        isUsed = (not "非自立" in node.feature) and isUsed
        isUsed = (not "数" in node.feature) and isUsed
        isUsed = (not "人名" in node.feature) and isUsed
        if isUsed:
            word += " {0}".format(node.feature.split(",")[6])
        '''
        if isUsed:
            if ("名詞接続" in pre_feature and "名詞" in node.feature) or ("接尾" in node.feature):
            word += "{0}".format(node.surface)
        else:
        word += " {0}".format(node.surface)
        #print("{0}{1}".format(node.surface, node.feature))
        '''
        pre_feature = node.feature
        node = node.next
    return word[1:]

In [12]:
def create_traindata(data_path, stopwords_path, save_path):
    ## csvの読み込み
    #df = pd.read_csv(data_path, encoding='utf-8', dtype={'Rating Score':'float'})
    df = pd.read_csv(data_path, encoding='utf-8')
    
    ## lists of reviews and titles
    reviews_list = df['reviews'].T.tolist()
    titles_list = df['title'].T.tolist()
    
    ## stopword
    stopwords_df = pd.read_csv(stopwords_path, encoding='utf-8')
    stopwords_list = stopwords_df.T.values.tolist()[0]
    stopwords_list.append("*")
    
    ## make training data
    splitted_reviews = [analysis(reviews).split(' ') for i, reviews in enumerate(reviews_list)]
    without_stopwords = []
    for i in range(len(splitted_reviews)):
        without_stopwords.append([word for word in splitted_reviews[i] if word not in stopwords_list])
    
    trainings = ""

    for i in range(len(without_stopwords)):
        if not without_stopwords[i]:
            continue
        trainings += " __label__" + str(i) + ", " + ' '.join(without_stopwords[i]) + "\n"
    # print(trainings)
    
    with open(save_path, mode='w') as f:
       f.write(trainings)

In [13]:
def make_model(trainings_path, myFastText_path):
    # save model
    model = fasttext.train_supervised(input=trainings_path, epoch=2000, dim=300, loss="hs")
    model.save_model(myFastText_path)

In [14]:
def myFastText(data_path, stopwords_path, myFastText_path):
    create_traindata(data_path, stopwords_path, 'data/testdata.txt')
    make_model('data/testdata.txt', myFastText_path)

In [74]:
myFastText('data/1005data.csv', 'data/stopwords.csv', 'data/model')

In [15]:
def predict(data_path, model_path, word, k):
    
    df = pd.read_csv(data_path, encoding='utf-8')
    titles_list = df['title'].T.tolist()
    
    model = fasttext.load_model(model_path)
    result = model.predict(word, k)
    for i in range(min([k, len(result[0])])):
        label = int(result[0][i].replace("__label__" , "").replace("," , ""))
        print(str(label) + " : " + titles_list[label] + " : " + str(result[1][i]))

In [16]:
predict('data/1005data.csv', 'data/model', "帰りたい", 20)

163 : 幸福な囚人 : 0.061527930200099945
275 : 漫画誕生 : 0.060788843780756
758 : シュヴァルの理想宮 ある郵便配達員の夢 : 0.06016926467418671
357 : M／村西とおる狂熱の日々　完全版 : 0.05852575972676277
306 : トラップ　劇場版 : 0.0576302707195282
899 : こおろぎ : 0.05676787719130516
892 : エッシャー　視覚の魔術師 : 0.05672869458794594
504 : この世界の（さらにいくつもの）片隅に : 0.05635904520750046
160 : 王と道化師たち : 0.05532108619809151
759 : ワイン・コーリング : 0.05531885847449303
888 : エッシャー　視覚の魔術師 : 0.054669853299856186
50 : ジョン・デロリアン : 0.0541025772690773
826 : 007／ノー・タイム・トゥ・ダイ : 0.053326208144426346
739 : こおろぎ : 0.05278072878718376
984 : 記憶屋 あなたを忘れない : 0.05259716138243675
960 : ドアロック : 0.05039467662572861
476 : アルファ　殺しの権利 : 0.04973023012280464
809 : スピード・スクワッド　ひき逃げ専門捜査班 : 0.04746798053383827
71 : テルアビブ・オン・ファイア : 0.0008202571771107614
389 : プリズナーズ : 0.000422843499109149
